# Combining all data

In [ ]:
!pip install -q easysnowdata

In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio as rio
import matplotlib.pyplot as plt
import easysnowdata
import folium
import earthaccess

In [ ]:
bbox_gdf = gpd.read_file('grand_mesa.geojson')
sample_area_gdf = gpd.read_file('../../data/sample_area.geojson')
sample_area_proj_gdf = sample_area_gdf.to_crs('EPSG:32613')

In [ ]:
station_collection = easysnowdata.automatic_weather_stations.StationCollection(sortby_dist_to_geom=bbox_gdf)
station_collection.get_data('622_CO_SNTL',variables=['WTEQ'])
#station_collection.WTEQ.loc[slice('2019-10-01','2020-09-30')].plot(ax=ax) # snotel

In [ ]:
s2 = easysnowdata.remote_sensing.Sentinel2(
    bbox_input=sample_area_gdf,
    start_date="2022-04-01",
    end_date="2022-04-30",
    resolution=40,
    catalog_choice="planetarycomputer",
)

In [ ]:
s2.data

In [ ]:
s2.get_rgb()
s2.get_ndsi()

In [ ]:
earthaccess.login()

In [ ]:
mod10a1f = easysnowdata.remote_sensing.MODIS_snow(bbox_gdf, start_date='2022-04-01', end_date='2022-04-30', data_product='MOD10A1F')

In [ ]:
s1 = easysnowdata.remote_sensing.Sentinel1(
    bbox_input=bbox_gdf, start_date="2019-10-01", end_date="2020-09-30", resolution=80, remove_border_noise=False,
)

In [ ]:
s1_rtc_sample_ds = s1.data.rio.clip(sample_area_proj_gdf.geometry)
s1_rtc_sample_median_ds = s1_rtc_sample_ds.median(dim=['x','y']).compute()

In [ ]:
orbits = s1_rtc_sample_median_ds['sat:relative_orbit'].values
unique_orbits = np.unique(orbits)
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_orbits)))

f,ax=plt.subplots(figsize=(10,5))

for i, orbit in enumerate(unique_orbits):

    if orbit in [56,151]:
        continue
        
    orbit_data = s1_rtc_sample_median_ds.where(s1_rtc_sample_median_ds['sat:relative_orbit'] == orbit, drop=True)

    time_of_day = "~6pm" if orbit_data['sat:orbit_state'].values[0] == "ascending" else "~6am"
    
    # Plot VV
    ax.plot(orbit_data.time, orbit_data.vv, 
             color=colors[i], linestyle='-', label=f'VV Orbit {orbit} ({time_of_day})')
    
    # Plot VH
    ax.plot(orbit_data.time, orbit_data.vh, 
             color=colors[i], linestyle='--', label=f'VH Orbit {orbit} ({time_of_day})')

# Customize the plot
ax.set_xlabel('Time')
ax.set_ylabel('Backscatter (dB)')
ax.set_title('S1 RTC VV and VH Backscatter by Relative Orbit')
ax.legend()